<center>

---
---

# OLÉ x candl

</center>

---
---

This notebook shows how to feed `candl` [1] likelihoods into OLÉ.
It is an adaptation and generalisation of the `candl_act_class_nuts.py` and `candl_spt_class_nuts.py` examples written by Sven.
At the moment this works for the ACT likelihood, SPT support is experimental and not well tested.

*Note*: this tutorial uses `candl`, install it via `pip install candl-like`

[1] https://arxiv.org/abs/2401.13433, https://github.com/Lbalkenhol/candl

In [1]:
from jax import config
import jax.numpy as jnp
import jax

import time
import numpy as np
from functools import partial

import sys
sys.path.insert(0, "/Users/lennart/Research/software/OLE_candl_development")

# OLE
from OLE.theory import Theory
from OLE.likelihood import Likelihood
from OLE.sampler import EnsembleSampler, Sampler, NUTSSampler

# candl
import candl
import candl.data

# CLASS
import classy

# Initialise candl Likelihood

In [2]:
candl_like = candl.Like(candl.data.ACT_DR4_TTTEEE)

Successfully initialised candl likelihood 'ACT DR4 TT/TE/EE (Choi et al. 2020, Aiola et al. 2020)' (type: <class 'candl.likelihood.Like'>).
Data loaded from '/opt/homebrew/Cellar/python@3.10/3.10.14/envs/jax_env/lib/python3.10/site-packages/candl/data/ACT_DR4_CMB_only_v0/'.
Functional likelihood form: gaussian
--------------------------------------------------------------------------------
It will analyse the following spectra:

TT dxd       (40 bins, bin centres spanning ell = 600.3 - 4123.0)
TE dxd       (45 bins, bin centres spanning ell = 350.5 - 4123.0)
EE dxd       (45 bins, bin centres spanning ell = 350.3 - 4122.9)
TT wxw       (40 bins, bin centres spanning ell = 600.4 - 4122.3)
TE wxw       (45 bins, bin centres spanning ell = 350.5 - 4122.4)
EE wxw       (45 bins, bin centres spanning ell = 350.4 - 4122.4)
--------------------------------------------------------------------------------
A data model consisting of 1 transformations has been initialised.
The following transform

# Theory Class Definition

In [3]:
class MyTheory(Theory):
    def initialize(self, **kwargs):
        super().initialize(**kwargs)   

        # input parameters of the theory
        self.requirements = ['h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'logA']

        self.cosmo = classy.Class()

        self.class_settings = {'output':'tCl,pCl,lCl,mPk',
                        'lensing':'yes',
                        'l_max_scalars': candl_like.ell_max,#7925 for ACT DR4, 3200, #  for SPT3G 2018
                        'N_ur':3.048,
                        'output_verbose':1,
                        }
        return 
    

    def compute(self, state):
        # Compute the observable for the given parameters.
        class_input = self.class_settings.copy()
        class_input['A_s'] = 1e-10*np.exp(state['parameters']['logA'][0])
        class_input['h'] = state['parameters']['h'][0]
        class_input['n_s'] = state['parameters']['n_s'][0]
        class_input['omega_b'] = state['parameters']['omega_b'][0]
        class_input['omega_cdm'] = state['parameters']['omega_cdm'][0]
        class_input['tau_reio'] = state['parameters']['tau_reio'][0]

        if state['parameters']['tau_reio'][0] < 0.01:
            class_input['tau_reio'] = 0.01

        self.cosmo.set(class_input)
        self.cosmo.compute()

        cls = self.cosmo.lensed_cl(candl_like.ell_max)

        state['quantities']['tt'] = cls['tt']
        state['quantities']['ee'] = cls['ee']
        state['quantities']['te'] = cls['te']
        state['quantities']['bb'] = cls['bb']

        return state

# Likelihood Class Definition

In [5]:
class OleCandlLikelihood(Likelihood):
    """
    OLÉ x candl wrapper
    This is a thin wrapper for candl likelihoods in OLÉ.
    """

    def initialize(self, **kwargs):
        """
        Initialisation of the likelihood, called right before sampling begins.
        Supply the requested data set via 'candl_dataset' as a path to a data set '.yaml' file.
        Note, you can pass candl shortcuts here, i.e. candl.data.ACT_DR4_TTTEEE will work, provided you have imported candl.data
        """

        print(kwargs)

        # Instantiate candl likelihood
        # By default internal priors in candl are NOT cleared, but you can override this by passing clear_candl_priors = True
        clear_priors = False
        if "clear_candl_priors" in kwargs:
            if kwargs["clear_candl_priors"]:
                clear_priors = True
        if clear_priors:
            self.candl_like = candl.Like(kwargs["candl_dataset"], priors=[])
        else:
            self.candl_like = candl.Like(kwargs["candl_dataset"])
        
        # Grab required parameters (from data model and priors)
        self.input_keys = list(np.unique(self.candl_like.required_nuisance_parameters + self.candl_like.required_prior_parameters))

        # Grab spectrum conversion helper
        self.cl2dl = (2.7255e6**2) * self.candl_like.ells * (self.candl_like.ells + 1) / (2.0 * jnp.pi)

        return

    # @partial(jax.jit, static_argnums=(0,))
    def loglike(self, state):
        """
        Compute the loglikelihood for the given parameters.
        Assumes the spectra are supplied from 0 to the correct ell_max by the theory code.
        """

        # Grab calculated spectra, convert to Dl
        Dl = {'ell': self.candl_like.ells}
        for spec_type in self.candl_like.unique_spec_types:
            Dl[spec_type] = state['quantities'][spec_type.lower()][2:] * self.cl2dl
        candl_input = {"Dl": Dl}

        # Push required parameters into dictionary for candl, convert any names
        for key in self.input_keys:
            if key == 'tau':
                candl_input['tau'] = state['parameters']['tau_reio'][0]
            else:
                candl_input[key] = state['parameters'][key][0]
        
        # Hand off to candl
        loglike = self.candl_like.log_like(candl_input)

        return jnp.array([loglike])

# Configure the Emulator and the Sampler

In [9]:
emulator_settings = {
    # the number of data points in cache before the emulator is to be trained
    'min_data_points': 80,

    # name of the cache file
    #'cache_file': './output_clang_spt_nuts/cache.pkl',
    'cache_file': './output_clang_act_nuts/cache.pkl',

    # accuracy parameters for loglike:
    'quality_threshold_constant': 1.0,
    'quality_threshold_linear': 0.1,

    # related so sampler
    'explained_variance_cutoff': 0.9999,

    # cache criteria
    'dimensionality': 7,
    'N_sigma': 5.0,

    #'compute_data_covmat': True,
    'data_covmat_directory': './act_data_covmats',

    # output directory
    'output_directory': './output_clang_act_nuts',

    # M adapt # burn-in of NUTS
    'M_adapt': 200,

    # 'plotting_directory': './plots_sampler_clang_nuts',
    # 'testset_fraction': 0.1,
    'logfile': './output_clang_act_nuts/log.txt',

    'learning_rate': 0.1,
    'num_iters': 300,


}

In [10]:
# Load the sampler
my_sampler = NUTSSampler(debug=False)

# Define cosmological parameters
my_parameters = {'h': {'prior': {'min': 0.60, 'max': 0.80}, 
                       'ref': {'mean': 0.68, 'std': 0.01},
                       'proposal': 0.01,},
                    'n_s': {'prior': {'min': 0.9, 'max': 1.1}, 'ref': {'mean': 0.965, 'std': 0.005},
                            'proposal': 0.01,}, 
                    'omega_b': {'prior': {'min': 0.02, 'max': 0.024}, 'ref': {'mean': 0.0223, 'std': 0.0003},
                                'proposal': 0.0002,}, 
                    'omega_cdm': {'prior': {'min': 0.10, 'max': 0.14}, 'ref': {'mean': 0.120, 'std': 0.002},
                                  'proposal': 0.002,}, 
                    'tau_reio': {'prior': {'min': 0.01, 'max': 0.1}, 'ref': {'mean': 0.055, 'std': 0.01},
                                 'proposal': 0.01,},
                    'logA': {'prior': {'min': 2.8, 'max': 3.3}, 'ref': {'mean': 3.1, 'std': 0.05},
                             'proposal': 0.05},
}

# Nuisance parameters
# Helpful code for adding nuisance parameters from ACT and SPT candl likelihoods below
if "ACT DR4 TT/TE/EE" in candl_like.name:
    my_parameters['yp'] = {'prior': {'min': 0.9, 'max': 1.1}, 'ref': {'mean': 1.0, 'std': 0.0001}, 'proposal': 0.0001}
elif "SPT-3G 2018 TT/TE/EE" in candl_like.name:
    for par_name in candl_like.required_nuisance_parameters:
        for prior in candl_like.priors:
            if par_name in prior.par_names and par_name not in my_parameters:
                
                # Grab the prior central value and width
                prior_central_val = float(prior.central_value[prior.par_names.index(par_name)])
                prior_width = np.sqrt(np.diag(prior.prior_covariance)[prior.par_names.index(par_name)])
                
                # Add to Cobaya's parameters
                my_parameters[par_name] = {'prior':
                                                {'min': prior_central_val-10*prior_width,
                                                    'max': prior_central_val+10*prior_width},
                                                'ref': {'mean': prior_central_val, 'std': prior_width},
                                               'proposal': prior_width}


# Run OLÉ!

In [11]:
start = time.time()

my_sampler.initialize(theory=MyTheory(),# Theory code defined at the start
                      likelihood=OleCandlLikelihood(),# Likelihood wrapper defined at the start
                      parameters=my_parameters,# Parameters to sample
                      covmat = None,
                      **emulator_settings,
                      candl_dataset = candl.data.ACT_DR4_TTTEEE,# candl dataset to use
                      clear_candl_priors = False,# Whether to switch off candl internal priors or not
                      )

# Note the total run steps are   (nsteps * nwalkers * MPI_size)
n_steps = 10
my_sampler.run_mcmc(n_steps)

end = time.time()
print("Time elapsed: ", end - start)

{'covmat': None, 'min_data_points': 80, 'cache_file': './output_clang_act_nuts/cache.pkl', 'quality_threshold_constant': 1.0, 'quality_threshold_linear': 0.1, 'explained_variance_cutoff': 0.9999, 'dimensionality': 7, 'N_sigma': 5.0, 'data_covmat_directory': './act_data_covmats', 'output_directory': './output_clang_act_nuts', 'M_adapt': 200, 'logfile': './output_clang_act_nuts/log.txt', 'learning_rate': 0.1, 'num_iters': 300, 'candl_dataset': '/opt/homebrew/Cellar/python@3.10/3.10.14/envs/jax_env/lib/python3.10/site-packages/candl/data/ACT_DR4_CMB_only_v0/ACT_DR4_CMB_only.yaml', 'clear_candl_priors': False}


2024-06-14 12:56:35,405 - Emulator - INFO - Emulator initialized
2024-06-14 12:56:35,405 - Emulator - INFO - Emulator initialized
INFO:Emulator:Emulator initialized


Successfully initialised candl likelihood 'ACT DR4 TT/TE/EE (Choi et al. 2020, Aiola et al. 2020)' (type: <class 'candl.likelihood.Like'>).
Data loaded from '/opt/homebrew/Cellar/python@3.10/3.10.14/envs/jax_env/lib/python3.10/site-packages/candl/data/ACT_DR4_CMB_only_v0/'.
Functional likelihood form: gaussian
--------------------------------------------------------------------------------
It will analyse the following spectra:

TT dxd       (40 bins, bin centres spanning ell = 600.3 - 4123.0)
TE dxd       (45 bins, bin centres spanning ell = 350.5 - 4123.0)
EE dxd       (45 bins, bin centres spanning ell = 350.3 - 4122.9)
TT wxw       (40 bins, bin centres spanning ell = 600.4 - 4122.3)
TE wxw       (45 bins, bin centres spanning ell = 350.5 - 4122.4)
EE wxw       (45 bins, bin centres spanning ell = 350.4 - 4122.4)
--------------------------------------------------------------------------------
A data model consisting of 1 transformations has been initialised.
The following transform

2024-06-14 12:56:36,327 - NUTSSampler - INFO - Test pipeline:
2024-06-14 12:56:36,327 - NUTSSampler - INFO - Test pipeline:
INFO:NUTSSampler:Test pipeline:
2024-06-14 12:56:36,328 - NUTSSampler - INFO - {'parameters': {'h': Array([0.67668313], dtype=float64), 'n_s': Array([0.96488818], dtype=float64), 'omega_b': Array([0.02239795], dtype=float64), 'omega_cdm': Array([0.12288086], dtype=float64), 'tau_reio': Array([0.05097675], dtype=float64), 'logA': Array([3.05919545], dtype=float64), 'yp': Array([1.00002683], dtype=float64)}, 'quantities': {'tt': array([0.00000000e+00, 0.00000000e+00, 1.45779716e-10, ...,
       7.51440385e-22, 7.51254999e-22, 7.51467174e-22]), 'ee': array([0.00000000e+00, 0.00000000e+00, 4.12946077e-15, ...,
       1.35797552e-23, 1.35682517e-23, 1.35567922e-23]), 'te': array([ 0.00000000e+00,  0.00000000e+00,  3.65476220e-13, ...,
       -1.92772572e-23, -1.92574337e-23, -1.92358772e-23]), 'bb': array([0.00000000e+00, 0.00000000e+00, 2.58915368e-19, ...,
       8.5

{'input_parameters': ['h', 'n_s', 'omega_b', 'omega_cdm', 'tau_reio', 'logA'], 'covmat': None, 'min_data_points': 80, 'cache_file': './output_clang_act_nuts/cache.pkl', 'quality_threshold_constant': 1.0, 'quality_threshold_linear': 0.1, 'explained_variance_cutoff': 0.9999, 'dimensionality': 7, 'N_sigma': 5.0, 'data_covmat_directory': './act_data_covmats', 'output_directory': './output_clang_act_nuts', 'M_adapt': 200, 'logfile': './output_clang_act_nuts/log.txt', 'learning_rate': 0.1, 'num_iters': 300, 'candl_dataset': '/opt/homebrew/Cellar/python@3.10/3.10.14/envs/jax_env/lib/python3.10/site-packages/candl/data/ACT_DR4_CMB_only_v0/ACT_DR4_CMB_only.yaml', 'clear_candl_priors': False}


...

Sample 1/211, time: 3.831273
Emulator good enough
Testing time:  10.274938106536865
Sample 2/211, time: 3.450629
Testing time:  0.0018901824951171875
Sample 3/211, time: 0.006283
Testing time:  0.0017468929290771484
Sample 4/211, time: 0.044097
Emulator good enough
Testing time:  0.01721811294555664
Sample 5/211, time: 0.005939
Testing time:  0.0019779205322265625
Sample 6/211, time: 0.088708
Emulator good enough
Testing time:  0.015588045120239258
Sample 7/211, time: 0.006587
Testing time:  0.0018780231475830078
Sample 8/211, time: 0.045109
Emulator good enough
Testing time:  0.01596379280090332
Sample 9/211, time: 0.006802
Testing time:  0.0016818046569824219
Sample 10/211, time: 0.089374
Emulator good enough
Testing time:  0.01647210121154785
Sample 11/211, time: 0.006541
Testing time:  0.0016551017761230469
Sample 12/211, time: 0.093402
Emulator good enough
Testing time:  0.016272783279418945
Sample 13/211, time: 0.013379
Testing time:  0.0016858577728271484
Sample 14/211, time: 0.

2024-06-14 12:57:50,988 - NUTSSampler - INFO - Iteration 99: Mean ESS = 72.970141
2024-06-14 12:57:50,988 - NUTSSampler - INFO - Iteration 99: Mean ESS = 72.970141
INFO:NUTSSampler:Iteration 99: Mean ESS = 72.970141
2024-06-14 12:57:50,989 - NUTSSampler - INFO - Iteration 99: Mean ESS/sek = 5.969988
2024-06-14 12:57:50,989 - NUTSSampler - INFO - Iteration 99: Mean ESS/sek = 5.969988
INFO:NUTSSampler:Iteration 99: Mean ESS/sek = 5.969988


Sample 96/211, time: 0.026346
Emulator good enough
Testing time:  0.011642694473266602
Sample 97/211, time: 0.019879
Emulator good enough
Testing time:  0.010223865509033203
Sample 98/211, time: 0.044475
Emulator good enough
Testing time:  0.009782791137695312
Sample 99/211, time: 0.006617
Testing time:  0.0017011165618896484
Sample 100/211, time: 0.045834
Emulator good enough
Testing time:  0.009884834289550781
Sample 101/211, time: 0.025249
Emulator good enough
Testing time:  0.014904260635375977
Sample 102/211, time: 0.006706
Testing time:  0.001935720443725586
Sample 103/211, time: 0.118095
Emulator good enough
Testing time:  0.009165048599243164
Sample 104/211, time: 0.005968
Testing time:  0.001772165298461914
Sample 105/211, time: 0.018022
Emulator good enough
Testing time:  0.00979924201965332
Sample 106/211, time: 0.020512
Emulator good enough
Testing time:  0.009312152862548828
Sample 107/211, time: 0.017429
Emulator good enough
Testing time:  0.008950233459472656
Sample 108/

2024-06-14 12:57:54,235 - NUTSSampler - INFO - Iteration 199: Mean ESS = 135.797560
2024-06-14 12:57:54,235 - NUTSSampler - INFO - Iteration 199: Mean ESS = 135.797560
INFO:NUTSSampler:Iteration 199: Mean ESS = 135.797560
2024-06-14 12:57:54,236 - NUTSSampler - INFO - Iteration 199: Mean ESS/sek = 8.778151
2024-06-14 12:57:54,236 - NUTSSampler - INFO - Iteration 199: Mean ESS/sek = 8.778151
INFO:NUTSSampler:Iteration 199: Mean ESS/sek = 8.778151


Sample 196/211, time: 0.018326
Testing time:  0.0018439292907714844
Sample 197/211, time: 0.042612
Emulator good enough
Testing time:  0.009690999984741211
Sample 198/211, time: 0.011453
Emulator good enough
Testing time:  0.008888959884643555
Sample 199/211, time: 0.017469
Emulator good enough
Testing time:  0.008678197860717773
Sample 200/211, time: 0.005784
Testing time:  0.0017058849334716797
Sample 201/211, time: 0.049048
Emulator good enough
Testing time:  0.008792877197265625
Sample 202/211, time: 0.018053
Emulator good enough
Testing time:  0.012109041213989258
Sample 203/211, time: 0.019794
Emulator good enough
Testing time:  0.009094953536987305
Sample 204/211, time: 0.017620
Emulator good enough
Testing time:  0.009338140487670898
Sample 205/211, time: 0.006070
Testing time:  0.0016813278198242188
Sample 206/211, time: 0.018316
Emulator good enough
Testing time:  0.009119987487792969
Sample 207/211, time: 0.018046
Emulator good enough
Testing time:  0.008927106857299805
Samp